In [2]:
import requests, json, os, csv, argparse, time
import pandas as pd
from PIL import Image, ImageDraw
from tqdm import tqdm
import subprocess
from google.cloud import storage
from config import ENVIRONMENT, KD_ID, CLIENT_ID, CLIENT_SECRET, MAX_OUTPUTS, AUTH_URL, EXCHANGE_ENDPOINT, API_BASE_URL
import image_functions as imgf
import json
from io import BytesIO
import askGPT as ask_gpt
import re
import threading
from IPython.display import Markdown, display

query = "What is the minimum baseline for LDL-C or non-high density lipoprotein cholesterol level to optimize efficacy of Repatha?"
df_in = pd.DataFrame({"Question": [query]})

def replace_with_links(text, urls):
    # Regular expression to find the numbered references
    pattern = r"\[(\d+)\]"
    
    # Replacement function
    def repl(match):
        number = match.group(1)  # Extract the number from the match
        url = urls.get(number, "#")  # Get the URL for the number, default to "#" if not found
        return f'<a href="{url}" target="_blank" style="font-size: 10px;">{number}</a>'  # Return the HTML link
    
    # Replace all occurrences in the text
    return re.sub(pattern, repl, text)    

def send_to_another_webhook(query):
    # Define the URL of the second webhook
    second_webhook_url = 'https://us-east4-dto-pryon-chatbot-dkuv.cloudfunctions.net/image_call_DTO'

    # Prepare the data to send
    data = {
        'text': query
    }

    # Make the POST request in a separate thread
    def post_data():
        try:
            requests.post(second_webhook_url, json=data)
            print("Data sent to the second webhook successfully.")
        except Exception as e:
            print(f"Failed to send data to the second webhook: {e}")

    # Start the thread
    thread = threading.Thread(target=post_data)
    thread.start()


 

b =""
user_query = """
This is the a user query: {QUESTION} {CONTEXT}
If the query is in a foreign language, please translate it to English before processing, and provide the language of the original query in the formation below. 
Break the query apart into "DESIRED FORMAT:" and "QUERY:" to feed into an IRQA system. I want to isolate the question for better input without any formatting instructions. 
Optimize the query for key words, related terms, or vocabulary that would be better processed by an IRQA system. Keep it in semantic language. 
If there appears to be only the query and no desired format, pass as the DESIRED FORMAT: General reply under 100 words, but comprehensive with descriptive detail in a formatted response. 
Optimize this query as much as possible for a semantic search query to an IRQA AI system. As needed, add in key words related to the query to assist in retreival of relevant information
If you determine the query is chit-chat in nature, in the sense that it is not requesting information is making comments, etc, simply pass the original query into QUERY: with a (chit-chat) label appended to the end 

The DESIRED FORMAT should be what you infer the format of the response the user is requesting, this should be in sentence form. Desired format should never reference the subject 
or material in the question. It is a general instruction.

Only give me this in your reply:

DESIRED FORMAT: (insert what you infer are the formatting instructions)
QUERY: (optimized query here)
LANGUAGE: (language of the original query)

EXAMPLE 1:

"Write me a story about dogs"

DESIRED FORMAT: Write the output as a story
QUERY: What can you tell me about dogs
LANGUAGE: English

EXAMPLE 2:
"What is the best breed of dogs, give me a list"

INSTRUCTIONS: Write the output as a list
QUERY: What are the best breeds of dogs
LANGUAGE: English
"""

reply = ask_gpt.askGPT(query,"",user_query)
print(reply)

# Using regular expressions to find matches for DESIRED FORMAT and QUERY
d_format_match = re.search(r"DESIRED FORMAT: (.*)", reply)
query_match = re.search(r"QUERY: (.*)", reply)
language = re.search(r"LANGUAGE: (.*)", reply)

# Extracting the matched groups if they are found
d_format = d_format_match.group(1).strip() if d_format_match else ""
query_new = query_match.group(1).strip() if query_match else ""
language = language.group(1).strip() if language else ""



df_in = pd.DataFrame({"Question": [query_new]})

df_exchange, access_token = imgf.LeoE2E(df_in)
# Assuming df_exchange is your DataFrame


answer = str(df_exchange.get('best_1_text', default=pd.Series([None]))[0])

source1_title = str(df_exchange.get('best_1_content_display_name', pd.Series([None])).iloc[0])
source2_title = str(df_exchange.get('best_2_content_display_name', pd.Series([None])).iloc[0])
source3_title = str(df_exchange.get('best_3_content_display_name', pd.Series([None])).iloc[0])
source4_title = str(df_exchange.get('best_4_content_display_name', pd.Series([None])).iloc[0])
source5_title = str(df_exchange.get('best_5_content_display_name', pd.Series([None])).iloc[0])


contextans = str(df_exchange.get('best_1_aic', pd.Series([None])).iloc[0])
contextans2 = str(df_exchange.get('best_2_aic', pd.Series([None])).iloc[0])
contextans3 = str(df_exchange.get('best_3_aic', pd.Series([None])).iloc[0])
contextans4 = str(df_exchange.get('best_4_aic', pd.Series([None])).iloc[0])
contextans5 = str(df_exchange.get('best_5_aic', pd.Series([None])).iloc[0])


conf_1_value = df_exchange.get('best_1_score', pd.Series([None])).iloc[0]
conf_2_value = df_exchange.get('best_2_score', pd.Series([None])).iloc[0]
conf_3_value = df_exchange.get('best_3_score', pd.Series([None])).iloc[0]
conf_4_value = df_exchange.get('best_4_score', pd.Series([None])).iloc[0]
conf_5_value = df_exchange.get('best_5_score', pd.Series([None])).iloc[0]

conf_1 = float(conf_1_value) if conf_1_value is not None else None
conf_2 = float(conf_2_value) if conf_2_value is not None else None
conf_3 = float(conf_3_value) if conf_3_value is not None else None
conf_4 = float(conf_4_value) if conf_4_value is not None else None
conf_5 = float(conf_5_value) if conf_5_value is not None else None


URL = str(df_exchange.get('best_1_url', pd.Series([None])).iloc[0])
URL2 = str(df_exchange.get('best_2_url', pd.Series([None])).iloc[0])
URL3 = str(df_exchange.get('best_3_url', pd.Series([None])).iloc[0])
URL4 = str(df_exchange.get('best_4_url', pd.Series([None])).iloc[0])
URL5 = str(df_exchange.get('best_5_url', pd.Series([None])).iloc[0])

content_id_1 = df_exchange.get('best_1_content_id', pd.Series([None])).iloc[0]
image_page_1 = df_exchange.get('ans1_text_sentence_bbox_1_page', pd.Series([None])).iloc[0]

content_id_2 = df_exchange.get('best_2_content_id', pd.Series([None])).iloc[0]
image_page_2 = df_exchange.get('ans2_text_sentence_bbox_1_page', pd.Series([None])).iloc[0]

content_id_3 = df_exchange.get('best_3_content_id', pd.Series([None])).iloc[0]
image_page_3 = df_exchange.get('ans3_text_sentence_bbox_1_page', pd.Series([None])).iloc[0]

content_id_4 = df_exchange.get('best_4_content_id', pd.Series([None])).iloc[0]
image_page_4 = df_exchange.get('ans4_text_sentence_bbox_1_page', pd.Series([None])).iloc[0]

content_id_5 = df_exchange.get('best_5_content_id', pd.Series([None])).iloc[0]
image_page_5 = df_exchange.get('ans5_text_sentence_bbox_1_page', pd.Series([None])).iloc[0]



page1 = image_page_1
page2 = image_page_2
page3 = image_page_3
page4 = image_page_4
page5 = image_page_5

# Append page number to URL if it's a PDF
if URL.endswith('.pdf'):
    Source1 = f"{URL}#page={page1}"
else:
    Source1 = URL

# Append page number to URL2 if it's a PDF
if URL2.endswith('.pdf'):
    Source2 = f"{URL2}#page={page2}"
else:
    Source2 = URL2

# Append page number to URL3 if it's a PDF
if URL3.endswith('.pdf'):
    Source3 = f"{URL3}#page={page3}"
else:
    Source3 = URL3

# Append page number to URL4 if it's a PDF
if URL4.endswith('.pdf'):
    Source4 = f"{URL4}#page={page4}"
else:
    Source4 = URL4

# Append page number to URL5 if it's a PDF
if URL5.endswith('.pdf'):
    Source5 = f"{URL5}#page={page5}"
else:
    Source5 = URL5

# Create image URLs based on the query if the source URLs are PDFs
query_letters_only = ''.join([char for char in query if char.isalpha()])

if URL.endswith('.pdf'):
    URL = f'https://storage.googleapis.com/dto_pryon_images/{query_letters_only}1.png'
else:
    URL = URL

if URL2.endswith('.pdf'):
    URL2 = f'https://storage.googleapis.com/dto_pryon_images/{query_letters_only}2.png'
else:
    URL2 = URL2

if URL3.endswith('.pdf'):
    URL3 = f'https://storage.googleapis.com/dto_pryon_images/{query_letters_only}3.png'
else:
    URL3 = URL3

if URL4.endswith('.pdf'):
    URL4 = f'https://storage.googleapis.com/dto_pryon_images/{query_letters_only}4.png'
else:
    URL4 = URL4

if URL5.endswith('.pdf'):
    URL5 = f'https://storage.googleapis.com/dto_pryon_images/{query_letters_only}5.png'
else:
    URL5 = URL5

# Dictionary of URLs
urls = {
    "1": URL,
    "2": URL2,
    "3": URL3,
    "4": URL4,
    "5": URL5,
}


default_prompt = (
    f"You are a RAG generative tool used to smooth answers from context information and you are responding to the question: {{QUESTION}}. "
    "Here are 6 instructions you must follow for each response, with all responses being in a qell structured markdown format with lists, paragrah breaks, etc as approproate:"
    "FIRST INSTRUCTION: If you detect the question is chit-chat, please simply follow the chit-chat guide at the end. "
    "SECOND INSTRUCTION: Use only the information provided in the context below. Provide a summary response as closely as you can for the related information, IMPORTANT: Please supply supplemental information in addition to the direct answer to provide some context and furthur insights "
    f"THIRD INSTRUCTION: Keep responses concise (100 words or less), and these instructions: {d_format} and in this language: {language}."
    "FOURTH INSTRUCTION: Format the response as a summary unless otherwise noted by the user. "
    "FIFTH INSTRUCTION: After each sentence, if information from the context was used, append the sentence with [n] where n is the source number where the information resides, whole numbers only never use decimals: For example [1] is valid and [1.1] is invalid. Output example: Sentence one proxy data here using source 2 [1] and info used here from source 2 [2]. "
    "SIXTH INSTRUCTION: If the information is not found in the context data provided, apologize to the user give a general summary of what information you do have in the context of the question "
    "CONTEXT INFORMATION - USE ONLY THIS PLEASE TO FORMAT A RESPONSE. YOU ARE A RAG TOOL. DO NOT DEVIATE FROM THIS INFORMATION: "
    "{{CONTEXT}}."
    "END CONTEXT INFORMATION "
    f"CHIT CHAT GUIDE"
    "If Chit-chat was detected, ignore all the context information above. Take on the personal of a friendly AI assistant. Try to address any of the concernst in a friendly manner"
    "Do no make up any information, simple be a conversational agent with a personality of helpfulness and professionalism")



pryon_data = (
    f"Source 1 Information: {contextans}\n"
    f"Source 2 Information: {contextans2}\n"
    f"Source 3 Information: {contextans3}\n"
    f"Source 4 Information: {contextans4}\n"  # Added source 4 information
    f"Source 5 Information: {contextans5}\n"  # Added source 5 information
)

# Assuming ask_gpt.askGPT is a function call relevant in your codebase
g_answer = ask_gpt.askGPT(query, pryon_data, default_prompt)

#print(g_answer)

formatted_generative = replace_with_links(g_answer, urls)

sources_string = ""
source_order = []

# Find all occurrences of [1] to [5] and their order
for marker in ["[1]", "[2]", "[3]", "[4]", "[5]"]:  # Included markers for [4] and [5]
    if marker in g_answer:
        source_order.append((g_answer.index(marker), marker))

# Sort by order of occurrence
source_order.sort()

# Append the source links and titles to sources_string based on the occurrence of markers
for _, marker in source_order:
    if marker == "[1]":
        sources_string += f'<a href="{Source1}" target="_blank" style="font-size: 6px;">SOURCE 1</a><span style="font-size: 10px; color: grey;">  {source1_title}</span><br>'
    elif marker == "[2]":
        sources_string += f'<a href="{Source2}" target="_blank" style="font-size: 6px;">SOURCE 2</a><span style="font-size: 10px; color: grey;">  {source2_title}</span><br>'
    elif marker == "[3]":
        sources_string += f'<a href="{Source3}" target="_blank" style="font-size: 6px;">SOURCE 3</a><span style="font-size: 10px; color: grey;">  {source3_title}</span><br>'
    elif marker == "[4]":  # Handling for source 4
        sources_string += f'<a href="{Source4}" target="_blank" style="font-size: 6px;">SOURCE 4</a><span style="font-size: 10px; color: grey;">  {source4_title}</span><br>'
    elif marker == "[5]":  # Handling for source 5
        sources_string += f'<a href="{Source5}" target="_blank" style="font-size: 6px;">SOURCE 5</a><span style="font-size: 10px; color: grey;">  {source5_title}</span><br>'

# Remove the last <br> tag if sources_string is not empty
if sources_string.endswith("<br>"):
    sources_string = sources_string[:-4]

formatted_html = sources_string


#Adding a custom OOD function here... to change response. 
if source1_title.startswith("VA:"):
    dialogflow_cx_response ={
        "fulfillment_response": {
            "messages": [
            {
                "text": {
                "text": [answer]
                }
            }
            ]
        }
    }
elif len(sources_string) < 5:
    dialogflow_cx_response ={
        "fulfillment_response": {
            "messages": [
            {
                "text": {
                "text": [formatted_generative]
                }
            }
            ]
        }
    }
else:
    # richContent is where you can modify dialogflow messenger data. 
    dialogflow_cx_response = {
        "fulfillment_response": {
            "messages": [
                {
                    "payload": {
                        "richContent": [[
                                #HTML TEST
                                {
                                    "type": "html",
                                    "html": [formatted_generative]
                                },
                                {
                                    "type": "html",
                                    "html": [formatted_html]
                                }
                            ]
                        ]
                    }
                }
            ]
        }
    }

display(Markdown(g_answer))
print("Source 1: ",source1_title)
print("Source 2: ",source2_title)
print("Source 3: ",source3_title)

 

DESIRED FORMAT: Provide a detailed answer in a concise format.
QUERY: Minimum LDL-C or non-HDL cholesterol level for Repatha efficacy optimization.
LANGUAGE: English


100%|██████████| 1/1 [00:02<00:00,  2.72s/it]


Sum total API Exchange latency: 2.402 seconds
Overall runtime for this dataset: 2.402 seconds


For patients at very high risk of ASCVD, particularly those who have experienced a recent myocardial infarction (MI), the minimum baseline for initiating treatment with a PCSK9 inhibitor like Repatha® to optimize efficacy is an LDL-C level of 70 mg/dL or greater, or a non-high density lipoprotein cholesterol (non-HDL-C) level of 100 mg/dL or greater[2][3]. Adding a PCSK9 inhibitor is recommended after maximizing statin therapy, possibly in combination with ezetimibe[3]. This approach is supported by guidelines aimed at reducing future cardiovascular risk and improving patient outcomes after an acute coronary syndrome event[2][3].

Source 1:  amgen-s3-gco-document-search/General/field_rep_ai/Repatha Mod3_Dyslipidaemia.pdf
Source 2:  amgen-s3-gco-document-search/General/field_rep_ai/Recent MI 1 year JAMA.pdf
Source 3:  amgen-s3-gco-document-search/General/field_rep_ai/Latest FOURIER sub-analysis Recent MI.pdf


In [3]:
display(Markdown(contextans))

In their position paper, the IAS identifies total risk as the sum of the 
following risk factors:43  

   Elevated LDL  
   Cigarette smoking  
   Hypertension  
   Diabetes  
   Low levels of HDL-C  
   Positive family history for ASCVD  

The IAS’ position towards management of dyslipidaemia is focused on 
identification of optimal levels of atherogenic cholesterol and adjustment 
of the intensity of cholesterol-lowering therapy to long-term risk. The table 
shown here summarizes their recommendations.43  

For primary prevention of ASCVD, the IAS defines optimal levels of 
atherogenic cholesterol as follows:  

   LDL:  
<100 mg/dL (2.6 mmol/L)  
   The optimal LDL in patients with established ASCVD is <70 mg/dL.43  
   Non–HDL-C: 
<130 mg/dL (3.4 mmol/L)  

DYSLIPIDAEMIA MODULE 3 

These materials are for internal training use only and are not to be shown to healthcare professionals. 
Amgen Confidential 2015. All rights reserved. EUHQ-P-145-0315-103230 
49 

CHAPTER 4: DIAGNOSIS OF DYSLIPIDAEMIA  
AND GENERAL TREATMENT RECOMMENDATIONS 

IAS Recommendations for Therapy by Risk Level  

TNS} for Therapy Ne Risk Level
Risk level to 80 Low (<15%) Moderate
years of age (15%-29%)
Therapeutic intensity Moderate
Specific therapy Public health MLT + CLD
recommendation? optional?
Moderately high High (>45%)
(30%-44%)
Moderately high High
MLT + CLD MLT + CLD
consideration® indicated*

CLD = cholesterol-lowering drug, usually a statin; IAS = International Atherosclerosis Society; MLT = maximal lifestyle therapies.
2 Persons at low risk of ASCVD should be treated according to national recommendation for the general public. These recommendations should be in accord
with IAS recommendations for lifestyle therapies.
© CLD therapy is usually reserved for patients with high levels of atherogenic cholesterol.
© Statin therapy is widely recommended for this risk category, although it is not accepted in many countries because of cost considerations. If drugs are used,
the dose should be adequate to achieve optimal atherogenic cholesterol levels.
4 CLD therapy is usually indicated in this category. The dose should be adequate to achieve optimal atherogenic cholesterol levels.

In [4]:
display(Markdown(contextans2))

Based on the 2018 American Heart Association/American
College of Cardiology Multisociety Guideline on the Manage-
ment of Blood Cholesterol, patients with clinical ASCVD
are separated into 2 different subgroups: those at very high risk
vs not.1 The management of patients at very high risk in-
cludes a recommendation for adding a PCSK9 inhibitor
in patients with LDL-C levels of 70 mg/dL or greater or
non–HDL-C levels of 100 mg/dL or greater in addition to maxi-
mally tolerated statin plus ezetimibe therapy.1 Likewise,
the 2019 European Society of Cardiology and European
Atherosclerosis Society guidelines for the management of
dyslipidemias categorizes patients with MI as a very high-
risk group and recommends both a reduction of LDL-C of
50% or greater and an LDL-C target of less than 55 mg/dL
(1.4 mmol/L).10

Early and systematic optimization of lipid-lowering
therapy is a validated process outcome for measurement of
quality improvement in patients with MI.11 However, obser-
vational studies showed that underuse of high-intensity
lipid-lowering therapy remains a well-known gap in second-
ary prevention.12 The present analysis from the FOURIER
trial highlights the importance of ensuring an optimal pro-
cess of care after hospital discharge within this critical first
year after MI.13 In addition, the benefit of adding PCSK9 inhi-
bition in patients with recent ACS was strengthened with
data from the Evaluation of Cardiovascular Outcomes After
an Acute Coronary Syndrome During Treatment With Ali-
rocumab (ODYSSEY Outcomes) trial14 where the median
time between the index event from the time of randomiza-
tion was 2.6 months.

Limitations

Our study had limitations. We acknowledge that only 3.3% and
6.3% of patients were treated with ezetimibe prior to random-

In [ ]:
display(Markdown(contextans3))

Based on the 2018 American Heart Association/American
College of Cardiology Multisociety Guideline on the Manage-
ment of Blood Cholesterol, patients with clinical ASCVD
are separated into 2 different subgroups: those at very high risk
vs not.1 The management of patients at very high risk in-
cludes a recommendation for adding a PCSK9 inhibitor
in patients with LDL-C levels of 70 mg/dL or greater or
non–HDL-C levels of 100 mg/dL or greater in addition to maxi-
mally tolerated statin plus ezetimibe therapy.1 Likewise,
the 2019 European Society of Cardiology and European
Atherosclerosis Society guidelines for the management of
dyslipidemias categorizes patients with MI as a very high-
risk group and recommends both a reduction of LDL-C of
50% or greater and an LDL-C target of less than 55 mg/dL
(1.4 mmol/L).10

Early and systematic optimization of lipid-lowering
therapy is a validated process outcome for measurement of
quality improvement in patients with MI.11 However, obser-
vational studies showed that underuse of high-intensity
lipid-lowering therapy remains a well-known gap in second-
ary prevention.12 The present analysis from the FOURIER
trial highlights the importance of ensuring an optimal pro-
cess of care after hospital discharge within this critical first
year after MI.13 In addition, the benefit of adding PCSK9 inhi-
bition in patients with recent ACS was strengthened with
data from the Evaluation of Cardiovascular Outcomes After
an Acute Coronary Syndrome During Treatment With Ali-
rocumab (ODYSSEY Outcomes) trial14 where the median
time between the index event from the time of randomiza-
tion was 2.6 months.

Limitations

Our study had limitations. We acknowledge that only 3.3% and
6.3% of patients were treated with ezetimibe prior to random-

In [ ]:
 #GENERATE IMAGES AND SAVE TO GOOGLE CLOUD STORAGE
content_image_pairs = [
    (content_id_1, image_page_1),
    (content_id_2, image_page_2),
    (content_id_3, image_page_3),
    (content_id_4, image_page_4),
    (content_id_5, image_page_5)
]

print(content_image_pairs)

bucket_name = 'dto_pryon_images'  # Your bucket name
credentials_json = 'gcp_cred.json'  # Your GCP credentials path
query_letters_only = ''.join([char for char in query if char.isalpha()])

for i, (content_id, image_page) in enumerate(content_image_pairs, start=1):
    if image_page:
        image = Image.open(BytesIO(imgf.get_image(access_token, content_id, image_page, uncompressed=0)))
        image_with_highlights = imgf.highlighted_contextimage(df_exchange, image, str(i))
        imgf.upload_to_gcp(content_id, image_with_highlights, bucket_name, credentials_json, f'{query_letters_only}{i}')
